# This case study consists of 1000 marks and the solution file is expected to be submitted in the upcoming session for evaluation.

# Combining Datasets: Merge and Join

One essential feature offered by Pandas is its high-performance, in-memory join and merge operations.
If you have ever worked with databases, you should be familiar with this type of data interaction.
The main interface for this is the ``pd.merge`` function, and we'll see few examples of how this can work in practice.

For convenience, we will start by redefining the ``display()`` functionality from the previous section:

In [2]:
import pandas as pd
import numpy as np


## Specification of the Merge Key

We've already seen the default behavior of ``pd.merge()``: it looks for one or more matching column names between the two inputs, and uses this as the key.
However, often the column names will not match so nicely, and ``pd.merge()`` provides a variety of options for handling this.

## Example: US States Data

Merge and join operations come up most often when combining data from different sources.
Here we will consider an example of some data about US states and their populations.
The data files can be found at http://github.com/jakevdp/data-USstates/:

In [2]:
# Following are shell commands to download the data
! mkdir -p data
!cd data ; curl -O  https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv ; cd ..
!cd data ; curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv ; cd ..
!cd data ; curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv ; cd ..

The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.


In [3]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


Let's take a look at the three datasets, using the Pandas ``read_csv()`` function:

In [4]:
pop = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv')
areas = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv')
abbrevs = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv')



Given this information, say we want to compute a relatively straightforward result: rank US states and territories by their 2010 population density.
We clearly have the data here to find this result, but we'll have to combine the datasets to find the result.

We'll start with a many-to-one merge that will give us the full state name within the population ``DataFrame``.
We want to merge based on the ``state/region``  column of ``pop``, and the ``abbreviation`` column of ``abbrevs``.
We'll use ``how='outer'`` to make sure no data is thrown away due to mismatched labels.

## Merge the 2 dataframes, pop and abbrevs such that all rows in both tables are included. Drop any duplicate column in the merged dataframe.

In [5]:
pop.head(2)

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0


In [6]:
abbrevs.head(2)

,state,abbreviation
0,Alabama,AL
1,Alaska,AK


In [32]:
abbrevs = abbrevs.rename(columns={'abbreviation':'state/region'})
abbrevs.head(2)

,state,state/region
0,Alabama,AL
1,Alaska,AK


In [35]:
merged = pd.merge(pop,abbrevs,on="state/region",how="outer")
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Let's double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

## Look for any rows with null values in the new dataframe

In [36]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

## Print out the rows in the merged dataframe that has null values for population.

It appears that all the null population values are from Puerto Rico prior to the year 2000; this is likely due to this data not being available from the original source.

More importantly, we see also that some of the new ``state`` entries are also null, which means that there was no corresponding entry in the ``abbrevs`` key!
Let's figure out which regions lack this match:

In [37]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

We can quickly infer the issue: our population data includes entries for Puerto Rico (PR) and the United States as a whole (USA), while these entries do not appear in the state abbreviation key.
We can fix these quickly by filling in appropriate entries:

In [38]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

No more nulls in the ``state`` column: we're all set!

Now we can merge the result with the area data using a similar procedure.
Examining our results, we will want to join on the ``state`` column in both:

In [48]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


## Repeat the process of checking for nulls in the area field for each state in the merged dataframe and deal with them appropriately.

In [49]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

## Select the portion of the data corresponding with the year 2000, and the total population.

In [50]:
newFinal = final.loc[final['year'] == 2000, 'population']
newFinal

28        4452173.0
29        1122273.0
68         627963.0
69         190615.0
124       5160586.0
125       1373414.0
162       2678588.0
163        680378.0
220      33987977.0
221       9267089.0
258       4326921.0
259       1106676.0
316       3411777.0
317        842242.0
354        786373.0
355        194914.0
412        572046.0
413        114503.0
452      16047515.0
453       3654880.0
508       8227303.0
509       2176576.0
546       1213519.0
547        295352.0
604       1299430.0
605        370430.0
642      12434161.0
643       3244944.0
700       6091866.0
701       1574989.0
           ...     
1852     12284173.0
1853      2918850.0
1890      1050268.0
1891       248065.0
1948      4024223.0
1949      1010641.0
1986       755844.0
1987       202681.0
2044      5703719.0
2045      1399685.0
2082     20944499.0
2083      5906301.0
2140      2244502.0
2141       721686.0
2178       609618.0
2179       147549.0
2236      7105817.0
2237      1741420.0
2274      5910512.0


## Re-index the data on state and find population density and display by density in ascending order

The result is a ranking of US states plus Washington, DC, and Puerto Rico in order of their 2010 population density, in residents per square mile.
We can see that by far the densest region in this dataset is Washington, DC (i.e., the District of Columbia); among states, the densest is New Jersey.

We can also check the end of the list:

We see that the least dense state, by far, is Alaska, averaging slightly over one resident per square mile.

This type of messy data merging is a common task when trying to answer questions using real-world data sources.
I hope that this example has given you an idea of the ways you can combine tools we've covered in order to gain insight from your data!

In [61]:
c = final.loc[final['year'] == 2010]
c['populationDensity'] = c['population']/c['area (sq. mi)']
c = c.groupby('state').sum().sort_values('populationDensity', ascending=True)
c

C:\Users\vrshr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,year,population,area (sq. mi),populationDensity
state,,,,
United States,4020,383445851.0,0.0,0.000000
Alaska,4020,901770.0,1312850.0,1.373759
Wyoming,4020,699573.0,195636.0,7.151782
Montana,4020,1213819.0,294092.0,8.254689
North Dakota,4020,824523.0,141408.0,11.661617
South Dakota,4020,1019356.0,154242.0,13.217619
New Mexico,4020,2583745.0,243186.0,21.249126
Idaho,4020,1999679.0,167148.0,23.927047
Nebraska,4020,2289459.0,154716.0,29.595633
